In [6]:
import tensorflow as tf

print(tf.__version__)   

2.17.0


In [5]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU'))) 

Num GPUs Available:  1


Trial

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os

# data path
dataset_path = "G:\My Drive\Datasets\ "

#Split the dataset
# Create an ImageDataGenerator for each subset
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 20% for validation

# Training data generator
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training')  # Set as training data

# Validation data generator
validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation')  # Set as validation data

# Create a separate ImageDataGenerator for test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Test data generator
test_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')  # No subset for testing




5. Create training, validation, and testing splits manually:

In [ ]:
from sklearn.model_selection import train_test_split
import shutil

# Function to create directories
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Paths to save splits
base_dir = "G:\My Drive\Datasets\small_split"
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

create_dir(base_dir)
create_dir(train_dir)
create_dir(val_dir)
create_dir(test_dir)

for class_name in ['organic', 'recyclable']:
    # Create class directories
    create_dir(os.path.join(train_dir, class_name))
    create_dir(os.path.join(val_dir, class_name))
    create_dir(os.path.join(test_dir, class_name))

    # List all files in the class directory
    class_files = os.listdir(os.path.join(dataset_path, class_name))
    train_files, test_files = train_test_split(class_files, test_size=0.2, random_state=42)  # 20% for testing
    train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)  # 20% of remaining for validation (0.25 * 0.8 = 0.2)

    # Copy files to respective directories
    for file in train_files:
        shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(train_dir, class_name, file))

    for file in val_files:
        shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(val_dir, class_name, file))

    for file in test_files:
        shutil.copy(os.path.join(dataset_path, class_name, file), os.path.join(test_dir, class_name, file))


6. Load the manually split data

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary')

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary')


In [ ]:
train_generator.class_indices

In [ ]:
train_generator.image_shape

In [ ]:
test_generator.image_shape

Classifying

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.models import load_model

import warnings
warnings.filterwarnings('ignore')

Creating the model

In [ ]:
img_shape = (128,128,3)
img_size = (128,128)
batch_size = 32

# MobileNetV2
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,
                                               include_top=False,
                                               weights='imagenet')

Freeze layers

In [ ]:
for layer in base_model.layers:
     layer.trainable = False


**Model 1**
The model 1 consists of Flatten layer, then we added two Dense layers with one Dropout layer as well

In [ ]:
model = tf.keras.Sequential([base_model,
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(32, activation="relu"),
                             tf.keras.layers.Dropout(0.2),
                             tf.keras.layers.Dense(1, activation="sigmoid")
                             ])


model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

#Train for 10 epochs
steps_per_epoch=int(len(train_generator)/batch_size)
validation_steps = validation_generator.samples // validation_generator.batch_size

# history = model.fit(
#     train_generator, steps_per_epoch,
#     verbose=1
# )

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,

    epochs=10,
    verbose=1
)

Visualising training results

In [ ]:
# plot function
def plot_graphs(history, string):
  plt.plot(history.history[string])
  # plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()


plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

Prediction for Model 1:


In [ ]:
loss, accuracy = model.evaluate(test_generator, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

# save model
model.save("tf_model.h5")
